In [43]:
import torch
from torch import nn
from torchvision import datasets
from torch.utils.data import Dataset
from torchvision.transforms import ToTensor 
from torch.utils.data import DataLoader

In [44]:
train_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)


In [45]:
batch_size = 64
train_dl = DataLoader(train_data,batch_size)
test_dl = DataLoader(test_data,batch_size)

In [46]:
for x,y in test_dl:
    print("the shape of X:",x.shape)
    print("the shape of y:",y.shape)
    break
# height ve weightlerin çarpımı bizim input dimensionimiz olacak

the shape of X: torch.Size([64, 1, 28, 28])
the shape of y: torch.Size([64])


In [47]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

class NeuralNetworkModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.model = nn.Sequential(
            nn.Linear(in_features=28*28,out_features=512),
            nn.ReLU(),
            nn.Linear(in_features=512,out_features=512),
            nn.ReLU(),
            nn.Linear(in_features=512,out_features=10)
        )



    def forward(self,x):
        x = self.flatten(x)
        output = self.model(x)
        return output

model = NeuralNetworkModel().to(device)
print(model)

Using cuda device
NeuralNetworkModel(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (model): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [48]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(params=model.parameters(),lr=1e-3)

In [54]:
def train(dataloader,model,loss_fn,optimizer):
    size = len(dataloader.dataset)
    model.train()
    # features and labels have to be in the same device
    for batch,(x,y) in enumerate(dataloader):
        x,y = x.to(device), y.to(device)

        # Compute prediction error
        pred = model(x)
        loss = loss_fn(pred,y)

        #backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        
        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(x)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
    

In [50]:
def test(dataloader,model,loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss,correct = 0,0
    with torch.no_grad():
        for x,y in dataloader:
            x,y = x.to(device), y.to(device)
            pred = model(x)
            test_loss += loss_fn(pred,y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")


In [55]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dl,model,loss_fn,optimizer)
    test(test_dl,model,loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.309624  [   64/60000]
loss: 2.290961  [ 6464/60000]
loss: 2.271246  [12864/60000]
loss: 2.264795  [19264/60000]
loss: 2.239261  [25664/60000]
loss: 2.219515  [32064/60000]
loss: 2.230751  [38464/60000]
loss: 2.196655  [44864/60000]
loss: 2.198726  [51264/60000]
loss: 2.156978  [57664/60000]
Test Error: 
 Accuracy: 39.3%, Avg loss: 2.152850 

Epoch 2
-------------------------------
loss: 2.172700  [   64/60000]
loss: 2.158756  [ 6464/60000]
loss: 2.097419  [12864/60000]
loss: 2.104462  [19264/60000]
loss: 2.052057  [25664/60000]
loss: 1.996182  [32064/60000]
loss: 2.023066  [38464/60000]
loss: 1.946776  [44864/60000]
loss: 1.960769  [51264/60000]
loss: 1.866876  [57664/60000]
Test Error: 
 Accuracy: 48.9%, Avg loss: 1.874052 

Epoch 3
-------------------------------
loss: 1.920617  [   64/60000]
loss: 1.884297  [ 6464/60000]
loss: 1.763705  [12864/60000]
loss: 1.791616  [19264/60000]
loss: 1.684395  [25664/60000]
loss: 1.639802  [32064/600

In [56]:
torch.save(model.state_dict(),"MineModel.pth")
print("Saved model state to MineModel.pth")

Saved model state to MineModel.pth


In [57]:
model = NeuralNetworkModel().to(device)
model.load_state_dict(torch.load("MineModel.pth"))

<All keys matched successfully>

In [80]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x,y = test_data[2][0], test_data[2][1]
with torch.no_grad():
    x = x.to(device)
    pred = model(x)
    predicted,actual = classes[pred.argmax()],classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')
    

Predicted: "Trouser", Actual: "Trouser"
